In [ ]:
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessor as t
from sklearn.metrics import f1_score
import collections


In [ ]:
def get_handler(x):
    return [w for wrd in re.findall(r'(#[\w]+)|(@[\w]+)|({[\w]+})',x) for w in wrd if w != '']

In [ ]:
def sentimnt_label(x):
    #   0: Negative, 1: Neutral, 2: Positive, 3: Can't Tell
    if x==0:
        x='Negative'
    elif x==1:
        x='Neutral'
    elif x==2:
        x='Positive'
    elif x==3:
        x="Can't Tell"
    else:
        x=''
    return x


In [ ]:
def sentimnt_int(x):
    #   0: Negative, 1: Neutral, 2: Positive, 3: Can't Tell
    if x=='Negative':
        x=0
    elif x=='Neutral':
        x=1
    elif x=='Positive':
        x=2
    elif x=="Can't Tell":
        x=3
    else:
        x=''
    return x


In [ ]:
def twt_clean(x):
    # Manual Cleaning
    #x = re.sub(r'(#[\w]+)|(@[\w]+)|({[\w]+})','',x)

    #x = re.sub(r'#[a-zA-Z .,\/]+ ','',x)
    #x = re.sub(r'(#[\w]+)|(@[\w]+)|({[\w]+})','',x)
    x = x.replace("#", "")
    x = x.replace(r"&\w+;", "")
    x = re.sub(r'[^a-zA-Z .,]+','',x)
    
    
    HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
    SAD_EMO = r" (:'?[/|\(]) "
    
    x = x.replace(HAPPY_EMO, " happyemoticons ")
    x = x.replace(SAD_EMO, " sademoticons ")

    # Library based Clean
    t.set_options(t.OPT.URL,t.OPT.RESERVED,t.OPT.MENTION)
    x=t.clean(x)
    x = x.lower()
    return x


In [ ]:
def tweet_token(x):
    tweet_tokenizer = TweetTokenizer()
    return tweet_tokenizer.tokenize(x)


In [ ]:
def stop_clean(x):
    manual_stopword_list = ['``', "'s", "...", "n't","sxsw","link","rt","quot","vs","ipad","iphone","google","apple","phone","quot","twitter","app","amp"]
    stop_words = list(set(stopwords.words('english')))+list(punctuation)+manual_stopword_list 
    res = []
    for i in x:
        if i not in stop_words and len(i)>2:
            res.append(i)
    return res            


In [ ]:
def stem_fun(x):
    stemmer = SnowballStemmer("english")
    res = []
    for i in x:
        res.append(stemmer.stem(i))
    return res

In [ ]:
def gen_wordcloud(text):
    wordcloud = WordCloud(
        width = 300,
        height = 200,
        background_color = 'black').generate(str(text))
        
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    return plt


In [ ]:
def model2(X,y,test):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.linear_model import LogisticRegression

    count_vec = CountVectorizer(ngram_range=(1, 1),stop_words = 'english',max_features=7000)    
    X_c = count_vec.fit_transform(X)
    X_c = X_c.toarray()    
    test_t = count_vec.transform(test)
    test_t = test_t.toarray()

    #X_train, X_test, y_train, y_test = train_test_split(X_c, y, test_size=0.10, random_state=0)
    
    clf_model = LogisticRegression(max_iter=300)
    clf = clf_model.fit(X_c, y)
    #acc = clf.score(X_test, y_test)
    #y_pred = clf.predict(X_test)    
    test_pred = clf.predict(test_t)    
    #f1 = f1_score(y_test, y_pred,average='micro')
    #print ('Model2 Accuracy: {}'.format(f1))
    return test_pred


In [ ]:
def data_load(file_path,_type):
    
    '''
    # Data Loading
    '''
    data_frame = pd.read_csv(file_path)

    data_frame.head()
    data_frame.columns  
    
    data_frame['tweet_len'] = data_frame['tweet'].str.len()
    #sns.countplot(x= 'tweet_len',data = train_tweet)
    
    
    '''
    # EDA Part
        1) Drop rows where tweet is NA
        2) Clean invalid char and convert string to lower 
        3) Tokenize
        4) Stopword Removal
        5) Stemming Words
    
    '''
    
    if _type == 'train':
        data_frame.dropna(subset = ["tweet"], inplace=True)
        data_frame['sentiment_label'] = data_frame['sentiment'].apply(sentimnt_label)

    data_frame['tweet_token'] = data_frame['tweet']
    
    data_frame['tweet_handler'] = data_frame['tweet_token'].apply(get_handler)
    data_frame['tweet_token'] = data_frame['tweet_token'].apply(twt_clean)
    
    data_frame['tweet_token'] = data_frame['tweet_token'].apply(tweet_token)
    
    data_frame['tweet_token'] = data_frame['tweet_token'].apply(stop_clean)
    
    #train_tweet['tweet_token_word'] = train_tweet['tweet_token'].apply(stem_fun)
    data_frame['tweet_token_word'] = data_frame['tweet_token'].apply(lemma_fun)
    data_frame['tweet_token_text'] = data_frame['tweet_token_word'].apply(lambda x: ' '.join(x))
    
    '''
    Word Cloud
    '''
    data_frame['type']=_type
    #gen_wordcloud(pos_words)
    #gen_wordcloud(neg_words)
    return data_frame
    


# Load Train Data

In [ ]:
file_path='file/data/train.csv'
train_tweet = data_load(file_path,'train')
X = train_tweet['tweet_token_text']
y = train_tweet['sentiment_label']


# Load Test Data

In [ ]:
file_path='file/data/test.csv'
train_tweet = data_load(file_path,'test')
test = train_tweet['tweet_token_text']


# Main Function

'''For Calling different prediction model as gererating files'''

In [ ]:
def main(i):
    file_path='file/data/train.csv'
    tweet = data_load(file_path,'train')
    X = tweet['tweet_token_text']
    y = tweet['sentiment_label']
    
    file_path='file/data/test.csv'
    tweet = data_load(file_path,'test')
    test = tweet['tweet_token_text']
    resp = model2(X,y,test)
    df = pd.DataFrame(data=resp)
    df = pd.concat([tweet['tweet_id'],df],axis=1)

    df['sentiment'] = df[0].apply(sentimnt_int)

    df.drop(0,inplace=True,axis=1)

    df.to_csv('submission_'+str(i)+'.csv', index=False)
    


# Call Main Function

In [ ]:
main(i)